In [1]:
from mleap.analyze_results import AnalyseResults
from mleap.data import Data
from mleap.data.data import Data
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error
from mleap.experiments import Orchestrator
from mleap.data.estimators import instantiate_default_estimators
import os

/media/viktor/Data/PhD/mleap/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
data = Data()
input_io = data.open_hdf5('data/delgado.hdf5', mode='r')
out_io = data.open_hdf5('data/classification.hdf5', mode='a')

In [2]:
dts_names = os.listdir('data/trained_models')
orchest = Orchestrator(hdf5_input_io=input_io, 
                           hdf5_output_io=out_io,
                           dts_names=dts_names,
                           original_datasets_group_h5_path='/delgado_datasets')

In [3]:
instantiated_models = instantiate_default_estimators(estimators=['NeuralNetworkDeepClassifier'], verbose=1)
orchest.predict_all(trained_models_dir='data/trained_models', estimators=instantiated_models)

Predictions of estimator NeuralNetworkDeepClassifier on hepatitis stored in database
Predictions of estimator RandomForestClassifier on hepatitis stored in database
Predictions of estimator Lasso on hepatitis stored in database
Predictions of estimator SVC on hepatitis stored in database
Predictions of estimator BaggingClassifier on hepatitis stored in database
Predictions of estimator GradientBoostingClassifier on hepatitis stored in database
Predictions of estimator PassiveAggressiveClassifier on hepatitis stored in database
Predictions of estimator BernoulliNaiveBayes on hepatitis stored in database
Predictions of estimator LogisticRegression on hepatitis stored in database
Predictions of estimator BaselineClassifier on hepatitis stored in database
Predictions of estimator GaussianNaiveBayes on hepatitis stored in database
Predictions of estimator LassoLars on hepatitis stored in database
Predictions of estimator RidgeRegression on hepatitis stored in database
Predictions of estimat

Predictions of estimator BaggingClassifier on glass stored in database
Predictions of estimator GradientBoostingClassifier on glass stored in database
Predictions of estimator PassiveAggressiveClassifier on glass stored in database
Predictions of estimator BernoulliNaiveBayes on glass stored in database
Predictions of estimator LogisticRegression on glass stored in database
Predictions of estimator BaselineClassifier on glass stored in database
Predictions of estimator GaussianNaiveBayes on glass stored in database
Predictions of estimator LassoLars on glass stored in database
Predictions of estimator RidgeRegression on glass stored in database
Predictions of estimator NeuralNetworkDeepClassifier on low_res_spect stored in database
Predictions of estimator RandomForestClassifier on low_res_spect stored in database
Predictions of estimator Lasso on low_res_spect stored in database
Predictions of estimator SVC on low_res_spect stored in database
Predictions of estimator BaggingClassifier

Predictions of estimator BaggingClassifier on mammographic stored in database
Predictions of estimator GradientBoostingClassifier on mammographic stored in database
Predictions of estimator PassiveAggressiveClassifier on mammographic stored in database
Predictions of estimator BernoulliNaiveBayes on mammographic stored in database
Predictions of estimator LogisticRegression on mammographic stored in database
Predictions of estimator BaselineClassifier on mammographic stored in database
Predictions of estimator GaussianNaiveBayes on mammographic stored in database
Predictions of estimator LassoLars on mammographic stored in database
Predictions of estimator RidgeRegression on mammographic stored in database
Predictions of estimator NeuralNetworkDeepClassifier on magic stored in database
Predictions of estimator RandomForestClassifier on magic stored in database
Predictions of estimator Lasso on magic stored in database
Predictions of estimator SVC on magic stored in database
Predictions

Predictions of estimator NeuralNetworkDeepClassifier on dermatology stored in database
Predictions of estimator RandomForestClassifier on dermatology stored in database
Predictions of estimator Lasso on dermatology stored in database
Predictions of estimator SVC on dermatology stored in database
Predictions of estimator BaggingClassifier on dermatology stored in database
Predictions of estimator GradientBoostingClassifier on dermatology stored in database
Predictions of estimator PassiveAggressiveClassifier on dermatology stored in database
Predictions of estimator BernoulliNaiveBayes on dermatology stored in database
Predictions of estimator LogisticRegression on dermatology stored in database
Predictions of estimator BaselineClassifier on dermatology stored in database
Predictions of estimator GaussianNaiveBayes on dermatology stored in database
Predictions of estimator LassoLars on dermatology stored in database
Predictions of estimator RidgeRegression on dermatology stored in datab

Predictions of estimator NeuralNetworkDeepClassifier on breast_tissue stored in database
Predictions of estimator RandomForestClassifier on breast_tissue stored in database
Predictions of estimator Lasso on breast_tissue stored in database
Predictions of estimator SVC on breast_tissue stored in database
Predictions of estimator BaggingClassifier on breast_tissue stored in database
Predictions of estimator GradientBoostingClassifier on breast_tissue stored in database
Predictions of estimator PassiveAggressiveClassifier on breast_tissue stored in database
Predictions of estimator BernoulliNaiveBayes on breast_tissue stored in database
Predictions of estimator LogisticRegression on breast_tissue stored in database
Predictions of estimator BaselineClassifier on breast_tissue stored in database
Predictions of estimator GaussianNaiveBayes on breast_tissue stored in database
Predictions of estimator LassoLars on breast_tissue stored in database
Predictions of estimator RidgeRegression on bre

Predictions of estimator NeuralNetworkDeepClassifier on ecoli stored in database
Predictions of estimator RandomForestClassifier on ecoli stored in database
Predictions of estimator Lasso on ecoli stored in database
Predictions of estimator SVC on ecoli stored in database
Predictions of estimator BaggingClassifier on ecoli stored in database
Predictions of estimator GradientBoostingClassifier on ecoli stored in database
Predictions of estimator PassiveAggressiveClassifier on ecoli stored in database
Predictions of estimator BernoulliNaiveBayes on ecoli stored in database
Predictions of estimator LogisticRegression on ecoli stored in database
Predictions of estimator BaselineClassifier on ecoli stored in database
Predictions of estimator GaussianNaiveBayes on ecoli stored in database
Predictions of estimator LassoLars on ecoli stored in database
Predictions of estimator RidgeRegression on ecoli stored in database
Predictions of estimator NeuralNetworkDeepClassifier on lung_cancer stored

Predictions of estimator NeuralNetworkDeepClassifier on led_display stored in database
Predictions of estimator RandomForestClassifier on led_display stored in database
Predictions of estimator Lasso on led_display stored in database
Predictions of estimator SVC on led_display stored in database
Predictions of estimator BaggingClassifier on led_display stored in database
Predictions of estimator GradientBoostingClassifier on led_display stored in database
Predictions of estimator PassiveAggressiveClassifier on led_display stored in database
Predictions of estimator BernoulliNaiveBayes on led_display stored in database
Predictions of estimator LogisticRegression on led_display stored in database
Predictions of estimator BaselineClassifier on led_display stored in database
Predictions of estimator GaussianNaiveBayes on led_display stored in database
Predictions of estimator LassoLars on led_display stored in database
Predictions of estimator RidgeRegression on led_display stored in datab

In [4]:
a = np.array([[1,2,3],[3,0,1]])

In [5]:
a.argmax(axis=1)

array([2, 0])